## Overview of Structured API Execution

1. Write DataFrame/Dataset/SQL Code.
2. If valid code, Spark converts this to a Logical Plan.
3. Spark transforms this Logical Plan to a Physical Plan, checking for optimizations along
the way.
4. Spark then executes this Physical Plan (RDD manipulations) on the cluster.

![Catalyst Optimizer](ct.svg)

### Logical Planning

This logical plan only represents a set of abstract transformations that do not refer to executors or
drivers, it’s purely to convert the user’s set of expressions into the most optimized version. It
does this by converting user code into an unresolved logical plan. This plan is unresolved
because although your code might be valid, the tables or columns that it refers to might or might
not exist. Spark uses the catalog, a repository of all table and DataFrame information, to resolve
columns and tables in the analyzer. The analyzer might reject the unresolved logical plan if the
required table or column name does not exist in the catalog. If the analyzer can resolve it, the
result is passed through the Catalyst Optimizer, a collection of rules that attempt to optimize the
logical plan by pushing down predicates or selections

![Logical Plan](lp.svg)

### Physical Planning

After successfully creating an optimized logical plan, Spark then begins the physical planning process. The physical plan, often called a Spark plan, specifies how the logical plan will execute on the cluster by generating different physical execution strategies and comparing them through a cost model. An example of the cost comparison might be choosing how to perform a given join by looking at the physical attributes of a given table

![Physical Plan](pl.svg)

### Execution

Upon selecting a physical plan, Spark runs all of this code over RDDs, the lower-level
programming interface of Spark (which we cover in Part III). Spark performs further
optimizations at runtime, generating native Java bytecode that can remove entire tasks or stages
during execution. Finally the result is returned to the user.

## Basic Structured Operations

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [2]:
df = spark.read.option("inferSchema", "true").options(header=True).csv("2015-summary.csv")

In [78]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)



### Schemas

In [79]:
df.schema

StructType([StructField('DEST_COUNTRY_NAME', StringType(), True), StructField('ORIGIN_COUNTRY_NAME', StringType(), True), StructField('count', IntegerType(), True)])

In [93]:
# Manually define schema
from pyspark.sql.types import StructField, StructType, StringType, IntegerType
schema = StructType([
StructField("DEST_COUNTRY_NAME", StringType(), True),
StructField("ORIGIN_COUNTRY_NAME", StringType(), True),
StructField("count", IntegerType())
])
schema

StructType([StructField('DEST_COUNTRY_NAME', StringType(), True), StructField('ORIGIN_COUNTRY_NAME', StringType(), True), StructField('count', IntegerType(), True)])

In [94]:
df = spark.read.options(header=True).schema(schema).csv("2015-summary.csv")

In [102]:
df

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: int]

### Columns and Expressions

#### Columns

Columns in Spark are similar to columns in a spreadsheet, R dataframe, or pandas DataFrame.
You can select, manipulate, and remove columns from DataFrames and these operations are
represented as expressions.
To Spark, columns are logical constructions that simply represent a value computed on a perrecord
basis by means of an expression. This means that to have a real value for a column, we
need to have a row; and to have a row, we need to have a DataFrame. You cannot manipulate an
individual column outside the context of a DataFrame; you must use Spark transformations
within a DataFrame to modify the contents of a column.

In [11]:
from pyspark.sql.functions import col, column

In [98]:
col('bigdata')

Column<'bigdata'>

In [100]:
column('spark')

Column<'spark'>

#### Expressions

- Columns are just expressions.- 
Columns and transformations of those columns compile to the same logical plan as
parsed expressions.

In [109]:
(((col("bigdata") + 24) * 3) - 88) < col("spark")

Column<'((((bigdata + 24) * 3) - 88) < spark)'>

In [113]:
from pyspark.sql.functions import expr
expr("((bigdata + 24) * 3 - 88) < spark")

Column<'((((bigdata + 24) * 3) - 88) < spark)'>

In [114]:
# Check dataframe columns
df.columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

### Records and Rows

In [115]:
df.first()

Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)

#### Creating Rows

In [116]:
from pyspark.sql import Row
myRow = Row("Hello", None, 1, False)

In [118]:
myRow[0]

'Hello'

### DataFrame Transformations

- We can add rows or columns
- We can remove rows or columns
- We can transform a row into a column (or vice versa)
- We can change the order of rows based on the values in columns

#### Creating DataFrames

In [125]:
from pyspark.sql import Row
from pyspark.sql.types import StructField, StructType, StringType, LongType
schema = StructType([
StructField("h1", StringType(), True),
StructField("h2", StringType(), True),
StructField("h3", LongType(), False)
])
Row1 = Row("Hello", None, 1)
Row2 = Row("Spark", 'data', 3)
mdf = spark.createDataFrame([Row1, Row2], schema)
mdf.show()

+-----+----+---+
|   h1|  h2| h3|
+-----+----+---+
|Hello|NULL|  1|
|Spark|data|  3|
+-----+----+---+



#### select and selectExpr

select and selectExpr allow you to do the DataFrame equivalent of SQL queries on a table of
data

In [15]:
df = spark.read.options(header=True, inferSchema=True).csv("2015-summary.csv")
df.createOrReplaceTempView("flights")

In [140]:
df.columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

In [137]:
# SQL
spark.sql('SELECT DEST_COUNTRY_NAME FROM flights LIMIT 2').show()

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+



In [139]:
# Dataframe
df.select('DEST_COUNTRY_NAME').limit(2).show()

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+



In [141]:
# SQL
spark.sql('SELECT ORIGIN_COUNTRY_NAME, DEST_COUNTRY_NAME FROM flights LIMIT 2').show()

+-------------------+-----------------+
|ORIGIN_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-------------------+-----------------+
|            Romania|    United States|
|            Croatia|    United States|
+-------------------+-----------------+



In [144]:
# Dataframe
df.select('ORIGIN_COUNTRY_NAME', 'DEST_COUNTRY_NAME').limit(2).show()

+-------------------+-----------------+
|ORIGIN_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-------------------+-----------------+
|            Romania|    United States|
|            Croatia|    United States|
+-------------------+-----------------+



In [145]:
df.select(['ORIGIN_COUNTRY_NAME', 'DEST_COUNTRY_NAME']).limit(2).show()

+-------------------+-----------------+
|ORIGIN_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-------------------+-----------------+
|            Romania|    United States|
|            Croatia|    United States|
+-------------------+-----------------+



In [152]:
df.select(col('ORIGIN_COUNTRY_NAME'), column('DEST_COUNTRY_NAME'), 'count').show()

+-------------------+--------------------+-----+
|ORIGIN_COUNTRY_NAME|   DEST_COUNTRY_NAME|count|
+-------------------+--------------------+-----+
|            Romania|       United States|   15|
|            Croatia|       United States|    1|
|            Ireland|       United States|  344|
|      United States|               Egypt|   15|
|              India|       United States|   62|
|          Singapore|       United States|    1|
|            Grenada|       United States|   62|
|      United States|          Costa Rica|  588|
|      United States|             Senegal|   40|
|      United States|             Moldova|    1|
|       Sint Maarten|       United States|  325|
|   Marshall Islands|       United States|   39|
|      United States|              Guyana|   64|
|      United States|               Malta|    1|
|      United States|            Anguilla|   41|
|      United States|             Bolivia|   30|
|           Paraguay|       United States|    6|
|      United States

In [153]:
# Change column name
# SQL
spark.sql('SELECT ORIGIN_COUNTRY_NAME as departure, DEST_COUNTRY_NAME as arrival FROM flights LIMIT 2').show()

+---------+-------------+
|departure|      arrival|
+---------+-------------+
|  Romania|United States|
|  Croatia|United States|
+---------+-------------+



In [162]:
# Dataframe
df.select(expr('ORIGIN_COUNTRY_NAME as departure'), expr('DEST_COUNTRY_NAME as arrival')).limit(2).show()

+---------+-------------+
|departure|      arrival|
+---------+-------------+
|  Romania|United States|
|  Croatia|United States|
+---------+-------------+



In [166]:
df.selectExpr('ORIGIN_COUNTRY_NAME as departure', 'DEST_COUNTRY_NAME as arrival').limit(2).show()

+---------+-------------+
|departure|      arrival|
+---------+-------------+
|  Romania|United States|
|  Croatia|United States|
+---------+-------------+



In [173]:
# Dataframe
df.selectExpr(
"*", # all original columns
"(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry")\
.show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [174]:
# SQL
spark.sql('SELECT f.*, f.DEST_COUNTRY_NAME = f.ORIGIN_COUNTRY_NAME as withinCountry FROM flights f LIMIT 2').show()

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+



In [177]:
# Dataframe
df.selectExpr("avg(count) as avg_count", "count(distinct(DEST_COUNTRY_NAME)) distinct_count").show(2)

+-----------+--------------+
|  avg_count|distinct_count|
+-----------+--------------+
|1770.765625|           132|
+-----------+--------------+



In [179]:
# SQL
spark.sql('SELECT avg(count) as avg_count, count(distinct(DEST_COUNTRY_NAME)) as distinct_count FROM flights LIMIT 2').show()

+-----------+--------------+
|  avg_count|distinct_count|
+-----------+--------------+
|1770.765625|           132|
+-----------+--------------+



#### Converting to Spark Types (Literals)

In [190]:
# Dataframe
from pyspark.sql.functions import lit
df.select(expr("*"), lit(1).alias("One")).show(2)

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



In [184]:
spark.sql('SELECT f.*, 1 as One FROM flights f LIMIT 2').show()

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+



#### Adding Columns

In [191]:
# Dataframe
df.withColumn("numberOne", lit(1)).show(2)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows



In [193]:
# SQL
spark.sql('SELECT *, 1 as numberOne FROM flights LIMIT 2').show()

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+



In [197]:
# Dataframe
df.withColumn("withinCountry", expr("ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME")).show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



#### Renaming Columns

In [210]:
df.withColumnRenamed("DEST_COUNTRY_NAME", "arrival")\
.withColumnRenamed("ORIGIN_COUNTRY_NAME", "departure")\
.show()

+--------------------+----------------+-----+
|             arrival|       departure|count|
+--------------------+----------------+-----+
|       United States|         Romania|   15|
|       United States|         Croatia|    1|
|       United States|         Ireland|  344|
|               Egypt|   United States|   15|
|       United States|           India|   62|
|       United States|       Singapore|    1|
|       United States|         Grenada|   62|
|          Costa Rica|   United States|  588|
|             Senegal|   United States|   40|
|             Moldova|   United States|    1|
|       United States|    Sint Maarten|  325|
|       United States|Marshall Islands|   39|
|              Guyana|   United States|   64|
|               Malta|   United States|    1|
|            Anguilla|   United States|   41|
|             Bolivia|   United States|   30|
|       United States|        Paraguay|    6|
|             Algeria|   United States|    4|
|Turks and Caicos ...|   United St

#### Case Sensitivity

By default Spark is case insensitive; however, you can make Spark case sensitive by setting the
configuration

In [6]:
#spark = SparkSession.builder.config('spark.sql.caseSensitive', 'true').getOrCreate()

In [7]:
df.select('dest_country_name').limit(2).show()

+-----------------+
|dest_country_name|
+-----------------+
|    United States|
|    United States|
+-----------------+



#### Removing Columns

In [9]:
df.drop('ORIGIN_COUNTRY_NAME', 'DEST_COUNTRY_NAME').columns

['count']

#### Changing a Column’s Type (cast)

In [ ]:
df.withColumn("count2", col("count").cast("long"))

In [16]:
spark.sql('SELECT *, cast(count as long) AS count2 FROM flights')

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: int, count2: bigint]

#### Filtering Rows

In [17]:
# Dataframe
df.filter(col("count") < 2).show(2)
df.where("count < 2").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [19]:
# SQL
spark.sql('SELECT * FROM flights WHERE count < 2 LIMIT 2').show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+



In [20]:
# Dataframe
df.where(col("count") < 2).where(col("ORIGIN_COUNTRY_NAME") != "Croatia")\
.show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [22]:
# SQL
spark.sql('SELECT * FROM flights WHERE count < 2 AND ORIGIN_COUNTRY_NAME != "Croatia" LIMIT 2').show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+



#### Getting Unique Rows

In [23]:
# Dataframe
df.select("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").distinct().count()

256

In [25]:
# SQL
spark.sql('SELECT COUNT(DISTINCT(ORIGIN_COUNTRY_NAME, DEST_COUNTRY_NAME)) as count FROM flights').show()

+-----+
|count|
+-----+
|  256|
+-----+



In [26]:
# Dataframe
df.select("ORIGIN_COUNTRY_NAME").distinct().count()

125

In [29]:
# SQL
spark.sql('SELECT COUNT(DISTINCT ORIGIN_COUNTRY_NAME) as count FROM flights').show()

+-----+
|count|
+-----+
|  125|
+-----+



#### Concatenating and Appending Rows (Union)

In [30]:
schema = df.schema

In [33]:
from pyspark.sql import Row
newRows = [
Row("New Country", "Other Country", 5),
Row("New Country 2", "Other Country 3", 1)
]

In [34]:
newRows

[<Row('New Country', 'Other Country', 5)>,
 <Row('New Country 2', 'Other Country 3', 1)>]

In [43]:
newDF = spark.createDataFrame(newRows, schema)

In [44]:
newDF.show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|      New Country|      Other Country|    5|
|    New Country 2|    Other Country 3|    1|
+-----------------+-------------------+-----+



In [46]:
df.union(newDF).where(col('DEST_COUNTRY_NAME').like('%New%')).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|      New Zealand|      United States|  111|
| Papua New Guinea|      United States|    3|
|    New Caledonia|      United States|    1|
|      New Country|      Other Country|    5|
|    New Country 2|    Other Country 3|    1|
+-----------------+-------------------+-----+



In [49]:
df.union(newDF).where(col('DEST_COUNTRY_NAME').contains('New')).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|      New Zealand|      United States|  111|
| Papua New Guinea|      United States|    3|
|    New Caledonia|      United States|    1|
|      New Country|      Other Country|    5|
|    New Country 2|    Other Country 3|    1|
+-----------------+-------------------+-----+



#### Sorting Rows

In [52]:
df.sort("count").show(5)

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
+--------------------+-------------------+-----+
only showing top 5 rows



In [53]:
df.orderBy("count", "DEST_COUNTRY_NAME").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
|           Cyprus|      United States|    1|
|         Djibouti|      United States|    1|
|        Indonesia|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [54]:
df.orderBy(col("count"), col("DEST_COUNTRY_NAME")).show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
|           Cyprus|      United States|    1|
|         Djibouti|      United States|    1|
|        Indonesia|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [57]:
from pyspark.sql.functions import desc, asc, expr

In [58]:
df.orderBy(expr("count desc")).show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Moldova|      United States|    1|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [59]:
df.orderBy(col("count").desc(), col("DEST_COUNTRY_NAME").asc()).show(2)

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
+-----------------+-------------------+------+
only showing top 2 rows



In [61]:
# SQL
spark.sql('SELECT * FROM flights ORDER BY count DESC, DEST_COUNTRY_NAME ASC LIMIT 2').show()

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
+-----------------+-------------------+------+



#### Limit

In [62]:
df.limit(5).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+



#### Collecting Rows to the Driver

In [76]:
collectDF = df.limit(30)

In [77]:
collectDF.take(5) # take works with an Integer count

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62)]

In [78]:
collectDF.show() # this prints it out nicely

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [74]:
collectDF.show(5, False)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
|United States    |Ireland            |344  |
|Egypt            |United States      |15   |
|United States    |India              |62   |
+-----------------+-------------------+-----+
only showing top 5 rows



In [75]:
collectDF.collect()

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Grenada', count=62),
 Row(DEST_COUNTRY_NAME='Costa Rica', ORIGIN_COUNTRY_NAME='United States', count=588),
 Row(DEST_COUNTRY_NAME='Senegal', ORIGIN_COUNTRY_NAME='United States', count=40),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]